In [1]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [2]:
from rdflib.namespace import Namespace, RDF, RDFS, XSD
from rdflib.term import URIRef, Literal
import csv
import json
import networkx as nx
import pandas as pd
import rdflib
from collections import defaultdict, Counter
import locale

In [3]:
graph = rdflib.Graph()
graph.parse('data/14_graph.nt', format='turtle')

<Graph identifier=Nd66e857ff2b74b51bc925581fea186c2 (<class 'rdflib.graph.Graph'>)>

In [4]:
# prefixes used in the graph
WD = Namespace('http://www.wikidata.org/entity/')
WDT = Namespace('http://www.wikidata.org/prop/direct/')
SCHEMA = Namespace('http://schema.org/')
DDIS = Namespace('http://ddis.ch/atai/')

In [20]:
# Read crowd data
crowd_df = pd.read_pickle("./data/crowd_data_preprocessed.pkl")
crowd_df

In [15]:
with open('./data/entity_ids.del', 'r') as ifile:
    ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
    id2ent = {v: k for k, v in ent2id.items()}
with open('./data/relation_ids.del', 'r') as ifile:
    rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
    id2rel = {v: k for k, v in rel2id.items()}
    
ent2lbl = {str(ent): str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

In [34]:
people_list = {str(uri):str(s) for uri,s in graph.query('''
PREFIX ddis: <http://ddis.ch/atai/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>
SELECT ?person ?lbl WHERE {
?person wdt:P31 wd:Q5 .
?person rdfs:label ?lbl .
}
''')}

len(people_list)

100157

In [35]:
movie_list = {str(uri):str(s) for uri,s in graph.query('''
PREFIX ddis: <http://ddis.ch/atai/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX schema: <http://schema.org/>
SELECT ?movie ?lbl WHERE {
?movie wdt:P31 wd:Q11424 .
?movie rdfs:label ?lbl .
}
''')}

len(movie_list)

24384

In [23]:
prefixes_dict={
    'wd' : 'http://www.wikidata.org/entity/',
    'wdt' : 'http://www.wikidata.org/prop/direct/',
    'schema' : 'http://schema.org/',
    'ddis' : 'http://ddis.ch/atai/',
}

In [27]:
predicates_set = dict()
entities_set = dict()

for idx, row in crowd_df.iterrows():
    h=row['Input1ID']
    r=row['Input2ID']
    t=row['Input3ID']
    h_word = ''
    r_word = ''
    t_word = ''
    try:
        prefix, ent = h.split(':')
        h_word = ent2lbl[prefixes_dict[prefix]+ent]
    except:
        h_word=h
    try:
        prefix, ent = r.split(':')
        r_word = ent2lbl[prefixes_dict[prefix]+ent]
    except:
        r_word=r
    try:
        prefix, ent = t.split(':')
        t_word = ent2lbl[prefixes_dict[prefix]+ent]
    except:
        t_word=t
    entities_set[h] = h_word
    entities_set[t] = t_word
    predicates_set[r] = r_word
    
    print(h_word + ' - ' + r_word + ' - ' + t_word)
        

Peaceful Warrior - box office - 4300000
God Help the Girl - publication date - 2014-01-18
Finding Nemo - box office - 969023261
The Browning Version - publication date - 1951-01-01
The Party's Just Beginning - publication date - 2018-02-24
The Princess and the Frog - box office - 267000000
The Twilight Saga: Eclipse - box office - 698491347
The Candidate - publication date - 2008-08-29
Nightjohn - publication date - 1996-06-01
station building - ddis:indirectSubclassOf - geographic entity
film organization - ddis:indirectSubclassOf - organization
fictional princess - ddis:indirectSubclassOf - fictional character
comics - ddis:indirectSubclassOf - written work
children's book - ddis:indirectSubclassOf - written work
literary pentalogy - ddis:indirectSubclassOf - series of creative works
supervillain team - ddis:indirectSubclassOf - fictional character
film organization - ddis:indirectSubclassOf - organization
neighbourhood of Helsinki - ddis:indirectSubclassOf - geographic entity
disput

In [28]:
predicates_set

{'wdt:P2142': 'box office',
 'wdt:P577': 'publication date',
 'ddis:indirectSubclassOf': 'ddis:indirectSubclassOf',
 'wdt:P161': 'cast member',
 'wdt:P495': 'country of origin',
 'wdt:P750': 'distributed by',
 'wdt:P136': 'genre',
 'wdt:.P344': 'wdt:.P344',
 'wdt:P725': 'voice actor',
 'wdt:P921': 'main subject',
 'wdt:P2554': 'production designer',
 'wdt:P58': 'screenwriter',
 'wdt:P3650': 'JMK film rating',
 'wdt:P3092': 'film crew member',
 'wdt:P364': 'original language of film or TV show',
 'wdt:P520': 'wdt:P520',
 'wdt:P3174': 'art director',
 'wdt:P945': 'allegiance',
 'wdt:P119': 'place of burial',
 'wdt:P276': 'location',
 'wdt:P20': 'place of death',
 'wdt:P1431': 'executive producer',
 'wdt:P272': 'production company',
 'wdt:P27': 'country of citizenship',
 'wdt:P1412': 'languages spoken, written or signed'}

In [33]:
'''MISING:
box office
'''

# Extra cases
# 1. Role
# 2. Genre

predicates_dict = {
    'publication date' : 'wdt:P577',
    'release date' : 'wdt:P577',
    'published' : 'wdt:P577',
    'released' : 'wdt:P577',
    'release' : 'wdt:P577',
    'when' : 'wdt:P577',
    'date' : 'wdt:P577',
    'year' : 'wdt:P577',
    
    
    # Subclass questions
    # 'is a' : 'ddis:indirectSubclassOf',
    # 'subclass' : 'ddis:indirectSubclassOf',
    # 'category' : 'ddis:indirectSubclassOf',
    
    # Country of origin
    'birthplace' : 'wdt:P495',
    'country of origin' : 'wdt:P495',
    'place of birth' : 'wdt:P495',
    'nationality' : 'wdt:P495',
    
    # Extra roles
    'main subject' : 'wdt:P921',
    'main character' : 'wdt:P921',
    'principal actor' : 'wdt:P921',
    'protagoinst': 'wdt:P921',
    'lead character': 'wdt:P921',
    'figure': 'wdt:P921',
    'production designer' : 'wdt:P2554',
    'executive producer' : 'wdt:P1431',
    
    # Specific ratings:
    'JMK' : 'wdt:P3650',
    'MPA' : 'wdt:P1657',
    'ICAA' : 'wdt:P3306',
    'FSK' : 'wdt:P1981',
    
    'original language' : 'wdt:P364',
    'language of origin' : 'wdt:P364',
    
    # Distribution
    'distributed' : 'wdt:P750',
    'distribution' : 'wdt:P750',
    'distributing' : 'wdt:P750',
    'distributor' : 'wdt:P750'
    
    'box office' : 'wdt:P2142',
    
    # Alliegance
    'allegiance' : 'wdt:P945',
    'fidelity' : 'wdt:P945',
    'loyalty': 'wdt:P945',
    'supporter': 'wdt:P945',
    'ally': 'wdt:P945',
    'allies': 'wdt:P945',
    'loyal to': 'wdt:P945',
    
    # Burial
    'place of burial' : 'wdt:P119',
    'buried' : 'wdt:P119',
    'burial location': 'wdt:P119',
    'burial site': 'wdt:P119',
    
    # Location
    'location' : 'wdt:P276',
    'located' : 'wdt:P276',
    
    # Place of death
    'place of death' : 'wdt:P20',
    'death' : 'wdt:P20',
    'died' : 'wdt:P20',
    
    # Production company
    'production company' : 'wdt:P272',
    'produced' : 'wdt:P272',
    
    # Country of citizenship
    'country of citizenship' : 'wdt:P27',
    'citizen' : 'wdt:P27',
    'citizenship' : 'wdt:P27',
    
    # Languages spoken written or signed
    'languages spoken' : 'wdt:P1412',
    'languages written' : 'wdt:P1412',
    'languages signed' : 'wdt:P1412',
    'languages spoken written or signed' : 'wdt:P1412',
    'speak': 'wdt:P1412',
    'spoken': 'wdt:P1412',
    'speaking' : 'wdt:P1412',
    'speaks' : 'wdt:P1412',
    'write': 'wdt:P1412',
    'writing' : 'wdt:P1412',
    'written': 'wdt:P1412',    
}

robusting_predicates = {
    "Recommend" : None,
    "visual" : None,
    "picture": None,
    "image": None,
    "photo": None
}

# is_a_member = {
#     station building
#     geographic entity
#     film organization
#     organization
#     fictional princess
#     fictional character
#     comics
#     written work
#     children's book
#     written work
#     literary pentalogy
#     series of creative works
#     supervillain team
#     fictional character
#     film organization
#     organization
#     neighbourhood of Helsinki
#     geographic entity
#     disputed territory
#     geographic entity
#     Silver Bear
#     award
# }

In [108]:
full_uri_dict = {}
for k,v in predicates_dict.items():
    prefix, i = v.split(':')
    full_uri_dict[k] = prefixes_dict[prefix]+i
print(full_uri_dict)

{'publication date': 'http://www.wikidata.org/prop/direct/P577', 'release': 'http://www.wikidata.org/prop/direct/P577', 'when': 'http://www.wikidata.org/prop/direct/P577', 'date': 'http://www.wikidata.org/prop/direct/P577', 'year': 'http://www.wikidata.org/prop/direct/P577', 'is a': 'http://ddis.ch/atai/indirectSubclassOf', 'subclass': 'http://ddis.ch/atai/indirectSubclassOf', 'category': 'http://ddis.ch/atai/indirectSubclassOf', 'birthplace': 'http://www.wikidata.org/prop/direct/P495', 'country of origin': 'http://www.wikidata.org/prop/direct/P495', 'place of birth': 'http://www.wikidata.org/prop/direct/P495', 'nationality': 'http://www.wikidata.org/prop/direct/P495', 'main subject': 'http://www.wikidata.org/prop/direct/P921', 'main character': 'http://www.wikidata.org/prop/direct/P921', 'principal actor': 'http://www.wikidata.org/prop/direct/P921', 'protagoinst': 'http://www.wikidata.org/prop/direct/P921', 'lead character': 'http://www.wikidata.org/prop/direct/P921', 'figure': 'http:

In [57]:
sentence_list_0 = [ # retrieve release date
    "Could you let me know the release date for <movie_name>?",
    "Could you tell me when <movie_name> was first shown?",
    "Give me the release date for <movie_name>",
    "I'd like to know the release date of the movie <movie_name>",
    "Please provide the release year for <movie_name>",
    "Tell me the release date of <movie_name>",
    "What's the date of release for <movie_name>?",
    "What's the release year of the movie <movie_name>?",
    "What's the launch date of <movie_name>?",
    "What's the debut date for <movie_name>?",
    "What is the release year of <movie_name>?",
    "When did <movie_name> come out?",
    "When did <movie_name> hit the theaters?",
    "When was <movie_name> released?",
    "When was <movie_name> launched?",
    "When was the movie <movie_name> released?",
    "When did <movie_name> first appear in cinemas?",
    "What is the publication date of <movie_name>",
    "Do you know the publication date of <movie_name>",
    "When was <movie_name> published?"
]

time_rel = {
    'publication date' : 'wdt:P577',
    'release date' : 'wdt:P577',
    'launch date' : 'wdt:P577',
    'published' : 'wdt:P577',
    'released' : 'wdt:P577',
    'launched' : 'wdt:P577',
    'when' : 'wdt:P577',
    'year' : 'wdt:P577',
}

i = 0
movies = list(movie_list.values())
res_s1 = []
for movie in movies:
    i+=1
    if(i==500):
        break
        
    sentence = sentence_list_0[i%len(sentence_list_0)]
    
    sentence = sentence.replace("<movie_name>",movie)

    ents = []
    for predicate in list(time_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))


    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s1.append((sentence,{"entities":entities}))
    



len(res_s1)

374

In [55]:
sentence_list_1 = [ # retrieve release date
    "What is the birthplace of <name>?",
    "Where was <name> born?",
    "Which is the country of origin of <name>?",
    "What is the nationality of <name>?",
    "What is the place of birth of <name>?",
    "In which city or town was <name> born?",
    "Tell me about the birthplace of <name>.",
    
]

origin_rel = {
    'birthplace' : 'wdt:P495',
    'country of origin' : 'wdt:P495',
    'place of birth' : 'wdt:P495',
    'nationality' : 'wdt:P495',
    'born': 'wdt:P495'
}

i = 0
names = list(people_list.values())
res_s2 = []
for name in names:
    i+=1
    if(i==200):
        break
        
    sentence = sentence_list_1[i%len(sentence_list_1)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(origin_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))


    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s2.append((sentence,{"entities":entities}))
    



len(res_s2)

199

In [60]:
sentence_list_3 = [ # retrieve release date
    "Give me the <role> of <movie_name>",
    "Give me the name of the <role> of <movie_name>",
    "I'd like to know the <role>'s name for <movie_name>",
    "I'm curious about the person who <directed/wrote> <movie_name>",
    "Identify the <role> of <movie_name>",
    "Tell me the <role> behind <movie_name>",
    "Tell me the <role> for <movie_name>",
    "Tell me the <role> of <movie_name>",
    "What is the <role>'s name for the movie <movie_name>?",
    "Who <role> in <movie_name>?",
    "Who <directed/wrote> <movie_name>?",
    "Who <directed/wrote> the movie <movie_name>?",
    "Who is the <role> of <movie_name>?",
    "Who is the <role> of the film <movie_name>?",
    "Who oversaw the making of <movie_name>",
    "Who was in charge of directing <movie_name>?",
    "Who was the <role> of <movie_name>"
    
]

origin_rel = {
    'main subject' : 'wdt:P921',
    'main character' : 'wdt:P921',
    'principal actor' : 'wdt:P921',
    'protagoinst': 'wdt:P921',
    'lead character': 'wdt:P921',
    'figure': 'wdt:P921',
    'production designer' : 'wdt:P2554',
    'executive producer' : 'wdt:P1431',
}

i = 0
roles = list(origin_rel.keys())
movies = list(movie_list.values())
res_s3 = []
for movie in movies:
    i+=1
    if(i==400):
        break
    
    sentence = sentence_list_3[i%len(sentence_list_3)]
    role = roles[i%len(roles)]
    
    sentence = sentence.replace("<movie_name>",movie).replace("<role>",role)

    ents = []
    ents.append((sentence.find(role),sentence.find(role) + len(role), "relation"))


    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s3.append((sentence,{"entities":entities}))
    



len(res_s3)

283

In [64]:
sentence_list_4 = [ # retrieve release date
    "What is the <rating> rating assigned to <movie>?",
    "Could you tell me the film rating given to <movie> by the <rating>?",
    "In terms of <rating> classification, what rating does <movie> hold?",
    "What <rating> film rating has been designated for <movie>?",
    "How has the <rating> rated the film <movie>?",
    "Tell me about the <rating> rating for <movie>.",
    "What classification has the <rating> given to <movie>?",
    "Can you inform me of the <rating> rating for the movie titled <movie>?",
    "In the context of <rating>, how is <movie> rated?",
    "What <rating> film classification does <movie> carry?",
    "What is the <rating> film rating of <movie>?"
    
]

rating_rel = {
    'JMK' : 'wdt:P3650',
    'MPA' : 'wdt:P1657',
    'ICAA' : 'wdt:P3306',
    'FSK' : 'wdt:P1981',
}

i = 0
ratings = list(rating_rel.keys())
movies = list(movie_list.values())
res_s4 = []
for movie in movies:
    i+=1
    if(i==400):
        break
    
    sentence = sentence_list_4[i%len(sentence_list_4)]
    rating = ratings[i%len(ratings)]
    
    sentence = sentence.replace("<movie>",movie).replace("<rating>",rating)

    ents = []
    ents.append((sentence.find(rating),sentence.find(rating) + len(rating), "association rating"))


    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s4.append((sentence,{"entities":entities}))
    



len(res_s4)

399

In [68]:
sentence_list_5 = [ # retrieve release date
    "What is the original language of <movie>",
    "What is the language of origin for <movie>?"
]

language_rel = {
    'original language' : 'wdt:P364',
    'language of origin' : 'wdt:P364',
}

i = 0
movies = list(movie_list.values())
res_s5 = []
for movie in movies:
    i+=1
    if(i==100):
        break
    
    sentence = sentence_list_5[i%len(sentence_list_5)]
    
    sentence = sentence.replace("<movie>",movie)

    ents = []
    for predicate in list(language_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s5.append((sentence,{"entities":entities}))
    



len(res_s5)

99

In [72]:
sentence_list_6 = [ # retrieve release date
    "Who distributed <movie>?",
    "Which company distributed <movie>?",
    "Which company handled the distribution of <movie>?",
    "Can you identify the distributor for <movie>?",
    "Who was responsible for distributing <movie>?",
    "What company took charge of distributing <movie>?",
    "In terms of distribution, which company is associated with <movie>?",
    "Which company managed the distribution of the film <movie>?",
    "Can you tell me the distribution company for <movie>?",
    "Who distributed the movie titled <movie>?",
    "Which distribution company was involved with <movie>?",
    "Tell me about the company responsible for the distribution of <movie>."
]

distribution_rel = {
    'distributed' : 'wdt:P750',
    'distribution' : 'wdt:P750',
    'distributing' : 'wdt:P750',
    'distributor' : 'wdt:P750'
}

i = 0
movies = list(movie_list.values())
res_s6 = []
for movie in movies:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_6[i%len(sentence_list_6)]
    
    sentence = sentence.replace("<movie>",movie)

    ents = []
    for predicate in list(distribution_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s6.append((sentence,{"entities":entities}))
    



len(res_s6)

299

In [77]:
sentence_list_7 = [ # retrieve release date
    "What is the box office of <movie>?",
    "How much revenue did <movie> generate at the box office?",
    "What is the box office earnings for the film <movie>?",
    "Can you provide information on the box office sales of <movie>?",
    "What was the financial success of <movie> at the box office?",
    "How did <movie> perform in terms of box office revenue?",
    "Tell me about the box office results for the movie titled <movie>.",
    "What were the box office numbers for <movie>?",
    "In terms of box office, how well did <movie> do?",
    "Can you share the box office figures for <movie>?",
    "What was the box office outcome for the film <movie>?",
    
]

i = 0
movies = list(movie_list.values())
res_s7 = []
for movie in movies:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_7[i%len(sentence_list_7)]
    
    sentence = sentence.replace("<movie>",movie)

    ents = []
    ents.append((sentence.find('box office'),sentence.find('box office') + len(predicate), "relation"))

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    res_s7.append((sentence,{"entities":entities}))
    



len(res_s7)

299

In [79]:
sentence_list_8 = [ # retrieve release date
    "Who is <name> loyal to?",
    "To whom belongs <name>'s loyalty",
    "What is the allegiance of <name>?",
    "What is the political allegiance of <name>?",
    "What is the fidelity of <name>",
    "What is <name> a supporter of?",
    "Which is the ally of <name>"
]

allegiance_rel = {
    'allegiance' : 'wdt:P945',
    'fidelity' : 'wdt:P945',
    'loyalty': 'wdt:P945',
    'supporter': 'wdt:P945',
    'ally': 'wdt:P945',
    'allies': 'wdt:P945',
    'loyal to': 'wdt:P945',
}

i = 0
names = list(people_list.values())
res_s8 = []
for name in names:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_8[i%len(sentence_list_8)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(allegiance_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s8.append((sentence,{"entities":entities}))
    



len(res_s8)

299

In [81]:
sentence_list_9 = [ # retrieve release date
    "What was the recording location of <movie>",
    "Where is the recording set of <movie> located?",
    "Can you provide information on the filming location of <movie>?",
    "Tell me about the location where <movie> was captured on film.",
    "Can you disclose the specific site where <movie> is located?",
    "What is the exact located position for the recording of <movie>?",
    "Where on the map can the recording of <movie> be located?",
    "Can you identify the exact located spot where <movie> was filmed?",
    "In terms of geography, where is <movie> located during filming?"
]

loc_rel = {
    'location' : 'wdt:P276',
    'located' : 'wdt:P276',
}

i = 0
movie = list(movie_list.values())
res_s9 = []
for movie in movies:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_9[i%len(sentence_list_9)]
    
    sentence = sentence.replace("<movie>",movie)

    ents = []
    for predicate in list(loc_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s9.append((sentence,{"entities":entities}))
    



len(res_s9)

299

In [89]:
sentence_list_10 = [ # retrieve release date
    "What is the place of death of <name>?",
    "Where <name> died?",
    "Where was <name> at the time of their death?",
    "Tell me about the site of <name>'s death.",
    "Can you specify where <name> died?",
    "Where was <name> located when they died?",
]

death_rel = {
    'place of death' : 'wdt:P20',
    'death' : 'wdt:P20',
    'died' : 'wdt:P20',
}

i = 0
names = list(people_list.values())
res_s10 = []
for name in names:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_10[i%len(sentence_list_10)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(death_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))
            break

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s10.append((sentence,{"entities":entities}))
    



len(res_s10)

299

In [90]:
sentence_list_11 = [ # retrieve release date
    "What is the place of burial of <name>?",
    "Where is <name> buried?",
    "Can you provide information about the place of burial of <name>?",
    "Can you provide information about the burial location of <name>?",
    "In what place is the burial site of <name> located?",
    "At which site is <name> buried?",
    "Can you specify where <name> is buried?"
]

death_rel = {
    'place of burial' : 'wdt:P119',
    'buried' : 'wdt:P119',
    'burial location': 'wdt:P119',
    'burial site': 'wdt:P119',
}

i = 0
names = list(people_list.values())
res_s11 = []
for name in names:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_11[i%len(sentence_list_11)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(death_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))
            break

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s11.append((sentence,{"entities":entities}))
    



len(res_s11)

299

In [92]:
sentence_list_12 = [ # retrieve release date
    "Which is the production company of <movie>?",
    "Which company produced <movie>?",
    "Who produced <movie>?",
    "What is the production company responsible for <movie>?",
    "Can you tell me which company was involved in the production of <movie>?",
    "Which company took charge of producing <movie>?",
    "Who is behind the production of <movie>?",
    "In terms of production, which company is associated with <movie>?",
    "Tell me about the entity that produced <movie>.",
    "What production company is credited for <movie>?",
    "Who played a role in the production of <movie>?",
    "Can you identify the production house for <movie>?"
]

production_rel = {
    'production company' : 'wdt:P272',
    'produced' : 'wdt:P272',
    'production' : 'wdt:P272',
    'producing' : 'wdt:P272'
}

i = 0
movies = list(movie_list.values())
res_s12 = []
for name in names:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_12[i%len(sentence_list_12)]
    
    sentence = sentence.replace("<movie>",movie)

    ents = []
    for predicate in list(production_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))
            break

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s12.append((sentence,{"entities":entities}))
    



len(res_s12)

299

In [95]:
sentence_list_13 = [ # retrieve release date
    "What is the country of citizenship of <name>?",
    "Which country is <name> a citizen of?",
    "What citizenship have <name>",
    "What country does <name> hold citizenship in?",
    "Of which country is <name> a citizen?",
    "Can you tell me the nationality or citizenship of <name>?",
    "Which country claims <name> as its citizen?",
    "To which nation does <name> belong in terms of citizenship?",
    "What is the country of citizenship for <name>?",
    "In what country is <name> recognized as a citizen?",
    "Which nation does <name> hold citizenship in?",
    "Can you specify the citizenship country of <name>?",
    "What is the citizenship status of <name>?"
]

citizen_rel = {
    'country of citizenship' : 'wdt:P27',
    'citizen' : 'wdt:P27',
    'citizenship' : 'wdt:P27',
}

i = 0
name = list(people_list.values())
res_s13 = []
for name in names:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_13[i%len(sentence_list_13)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(citizen_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))
            break

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s13.append((sentence,{"entities":entities}))
    



len(res_s13)

299

In [98]:
sentence_list_14 = [ # retrieve release date
    "What are the languages spoken by <name>?",
    "What are the languages written by <name>?",
    "What are the languages signed by <name>?",
    "What languages does <name> speak?",
    "What were the languages spoken written or signed in <name>?",
    "What were the languages spoken by <name>",
    "Which languages is <name> proficient in speaking?",
    "Can you list the languages that <name> speaks?",
    "Which languages does <name> have command over in terms of speaking?",
    "Can you provide information on the languages spoken by <name>?",
    "Which languages does <name> use for communication?",
    "Tell me about the linguistic skills of <name> in terms of speaking",
    "Tell me about the linguistic skills of <name> in terms of writing",
]

lan_rel = {
    'languages spoken' : 'wdt:P1412',
    'languages written' : 'wdt:P1412',
    'languages signed' : 'wdt:P1412',
    'languages spoken written or signed' : 'wdt:P1412',
    'speak': 'wdt:P1412',
    'spoken': 'wdt:P1412',
    'speaking' : 'wdt:P1412',
    'speaks' : 'wdt:P1412',
    'write': 'wdt:P1412',
    'writing' : 'wdt:P1412',
    'written': 'wdt:P1412',
}

i = 0
name = list(people_list.values())
res_s14 = []
for name in names:
    i+=1
    if(i==300):
        break
    
    sentence = sentence_list_14[i%len(sentence_list_14)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(lan_rel.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "relation"))
            break

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s14.append((sentence,{"entities":entities}))
    



len(res_s14)

276

In [100]:
sentence_list_15 = [
    # Movie recommendation
    # Three movies
    "Recommend movies like <name>",
    "Recommend me some films in the same vein as <name>.",
    "Recommend movies similar to <name>.",
    "Recommend some films like <name>.",
    "Recommend me some movies in the same vein as <name>.",
    "Recommend any good movies similar to <name>.",
    "Recommend films similar to <name> that I should watch.",
    
    "Show me a picture of <name>",
    "How does <name> looks like?",
    "Have you any images of <name>",
    "Can you display a photo of <name>",
    "What does <name> look like?",
    "Let me know what <name> looks like.",
    "Could you please show me an image of <name>?",
    "I'm curious about <name>'s appearance. Can you provide a picture?",
    "Do you have a photo of <name>?",
    "I'd like to see a photo of <name>",
    "Do you have any visuals or pictures related to <movie_name> that you can share?",

]

i = 0
name = list(people_list.values())
res_s15 = []
for name in names:
    i+=1
    if(i==600):
        break
    
    sentence = sentence_list_15[i%len(sentence_list_15)]
    
    sentence = sentence.replace("<name>",name)

    ents = []
    for predicate in list(robusting_predicates.keys()):
        if predicate in sentence:
            ents.append((sentence.find(predicate),sentence.find(predicate) + len(predicate), "predicate"))
            break

    entities = []
    for ent in ents:
        if(ent[0]!=-1):
            entities.append(ent)
    if len(entities) == 0:
        continue
    res_s15.append((sentence,{"entities":entities}))
    



len(res_s15)

500

In [101]:
res_s15

[('Recommend me some films in the same vein as Viktor Krištof.',
  {'entities': [(0, 9, 'predicate')]}),
 ('Recommend movies similar to Yuji Nomi.',
  {'entities': [(0, 9, 'predicate')]}),
 ('Recommend some films like Béatrice Thiriet.',
  {'entities': [(0, 9, 'predicate')]}),
 ('Recommend me some movies in the same vein as Oleg Kapanets.',
  {'entities': [(0, 9, 'predicate')]}),
 ('Recommend any good movies similar to Ram Lee.',
  {'entities': [(0, 9, 'predicate')]}),
 ('Recommend films similar to Gabriel Migliori that I should watch.',
  {'entities': [(0, 9, 'predicate')]}),
 ('Show me a picture of Alexander Geringas',
  {'entities': [(10, 17, 'predicate')]}),
 ('Have you any images of Marco Antônio Guimarães',
  {'entities': [(13, 18, 'predicate')]}),
 ('Can you display a photo of Mauro Senise',
  {'entities': [(18, 23, 'predicate')]}),
 ('Could you please show me an image of Pierre Volto?',
  {'entities': [(28, 33, 'predicate')]}),
 ("I'm curious about Carlo Siliotto's appearance. 

In [102]:
TRAIN_DATA = res_s1 + res_s2 + res_s3 + res_s4 + res_s5 + res_s6 + res_s7 + res_s8+res_s9+res_s10+res_s11+res_s12+res_s13+res_s14+res_s15
len(TRAIN_DATA)

4522

In [103]:
import pandas as pd
import os
from tqdm import tqdm
from spacy.tokens import DocBin

nlp = spacy.load("en_core_web_sm")

db = DocBin() # create a DocBin object

count = 0
for text, annot in tqdm(TRAIN_DATA): # data in previous format
    try:
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                #print("Skipping entity")
                count+=1
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    except Exception as e:
        print(text)
        print(annot)
        print(e)
        count+=1
print(count)
#os.chdir(r'XXXX\XXXXX')
db.to_disk("train_ner2.spacy") # save the docbin object

100%|████████████████████████████████████████████████████████████████████████████| 4522/4522 [00:00<00:00, 8646.28it/s]

320


In [106]:
nlp_model = spacy.load('models/NER2/loss_23/')

total=0
correct=0
for sentence, entities in TRAIN_DATA:
    pred = nlp_model(sentence)
    total+=len(entities["entities"])
    for ent in pred.ents:
        if (ent.start_char, ent.end_char, ent.label_) in entities['entities']:
            correct+=1
            #print(ent.label_+" Correctly found")
        else:
            spacy.displacy.render(pred, style="ent", jupyter=True)
            print(sentence, entities)
            print("Missing {} label".format(ent.label_))
            
print(correct, total)

What is the box office earnings for the film Moondram Pirai? {'entities': [(12, 23, 'relation')]}
Missing relation label


Can you provide information on the box office sales of Buffalo Bill and the Indians, or Sitting Bull's History Lesson? {'entities': [(35, 46, 'relation')]}
Missing relation label


How did Wanted: Dead or Alive perform in terms of box office revenue? {'entities': [(50, 61, 'relation')]}
Missing relation label


Tell me about the box office results for the movie titled Linger. {'entities': [(18, 29, 'relation')]}
Missing relation label


What were the box office numbers for Eastern Condors? {'entities': [(14, 25, 'relation')]}
Missing relation label


...

Can you share the box office figures for Ugetsu? {'entities': [(18, 29, 'relation')]}
Missing relation label


What was the box office outcome for the film Swing Kids? {'entities': [(13, 24, 'relation')]}
Missing relation label


What is the box office of Gallipoli? {'entities': [(12, 23, 'relation')]}
Missing relation label


What is the box office earnings for the film The Truce? {'entities': [(12, 23, 'relation')]}
Missing relation label


In what place is the burial site of Dikla located? {'entities': [(21, 32, 'relation')]}
Missing relation label
3986 4523
